# Calculate potential evapotranspiration (PET)

*******************************

Uses daily or monthly data from downscaled CMIP6 outputs to calculate mean daily ETrs and ETos, using the version of the Penman-Monteith expression recommended by the American Society of Civil Engineers Environmental Water Resources Institute (ACSCE-EWRI), and known as the ASCE Standardized Reference ET equation:

Reference: ASCE-EWRI (2005): The ASCE Standardized Reference Evapotranspiration Equation, Task Committee on Standardization of Reference Evapotranspiration, Ed. Richard G. Allen

Inputs, from mean daily values in BARPA/CCAM/WRF on Gadi: Elev = Elevation [m] Lat = Latitude [degs] Lon = Longitude [degs] SpHm = 2-metre specific humidity [kg/kg] Patm = Surface pressure [Pa] U2m = 2-metre U wind [m/sec] SWdn = Surface downward shortwave radiation flux [W/m^2] Tmax = Daily maximum 2-metre temperature [K] Tmin = Daily minimum 2-metre temperature [K]

Output: ETrs = ASCE-EWRI tall reference crop ET for 0.5 m alfalfa [mm/day] ETos = ASCE-EWRI short reference crop ET for 12 cm grass [mm/day]

All raw inputs, unit-converted inputs and outputs are in upper case. All intermediate variables are in lower case. All parameters are expressed in SI units.

History:
- Fortran code author: Mike Hobbins Date: December 19, 2013 Converted to Python: David Hoffmann, Monash University PhD Candidate Date: October 9, 2018
- Python code adapted for ERA5: Tess Parker, Monash University Research Fellow Date: November 2019
- Python code adapted for BARRA R2: Tess Parker, CSIRO Research Scientist Date: October 2023
- Python code adapted for downscaled CMIP6: David Hoffmann, Climate Scientist, Bureau of Meteorology. Date: January 2025

*******************************

In [1]:
import numpy as np
import time
import os
import pandas as pd
import datetime
import cf_units as unit
import matplotlib as plt
import sys
import xarray as xr
import math
from dask.distributed import Client
import tempfile
import dask
from dask.diagnostics import ProgressBar

import warnings 
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

In [3]:
# Dask settings
dask.config.set({
    #'array.chunk-size': "256 MiB",
    #'array.slicing.split_large_chunks': True, 
    'distributed.comm.timeouts.connect': '120s',
    'distributed.comm.timeouts.tcp': '120s',
    'distributed.comm.retry.count': 10,
    'distributed.scheduler.allowed-failures': 20,
    "distributed.scheduler.worker-saturation": 1.1, #< This should use the new behaviour which helps with memory pile up
})

enc = {"ETrs": {
        "zlib": True,
        "complevel": 1,
        "shuffle": True
        },
       "ETos": {
        "zlib": True,
        "complevel": 1,
        "shuffle": True
        }
      }
enc_H = {"pet_hargraves": {
        "zlib": True,
        "complevel": 1,
        "shuffle": True
            }
        }
       
client = Client(n_workers=10, threads_per_worker=1, local_directory = tempfile.mkdtemp(), memory_limit = "63000mb")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34439/status,
Dashboard: /proxy/34439/status,Workers: 10
Total threads: 10,Total memory: 586.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33235,Workers: 10
Dashboard: /proxy/34439/status,Total threads: 10
Started: Just now,Total memory: 586.73 GiB
Comm: tcp://127.0.0.1:42935,Total threads: 1
Dashboard: /proxy/38761/status,Memory: 58.67 GiB
Nanny: tcp://127.0.0.1:38505,


In [4]:
def extraTerrRadiation(tasmax, lat_name='lat'):
    # Day of year
    _jday = tasmax.time.dt.dayofyear.values #Jday
    jday = xr.DataArray(_jday, coords=[('day', _jday)], dims=["day"])
                    
    # Declination, decl [rad] = f(pi [-], jday [-])
    # Allen et al 2005 equation 51 solar declination
    decl = 0.409 * np.sin((2. * np.pi * jday / 365) - 1.39)
        
    # Sunset hour angle, ws [rad] = f(Lat [rad], decl [rad])
    # Allen et al 2005 equation 59 - is this correct? should it be tan of lat???
    Lat  = tasmax[lat_name]
    ws = np.arccos(-1 * np.sin(np.deg2rad(Lat)) * np.tan(decl))
        
    # Inverse relative distance of earth from sun, dr [-] = f(pi [-], jday [-]), Allen et al 2005 equation 50
    dr = 1. + 0.033 * np.cos(2. * np.pi * jday / 365)

    # Solar constant, Gsc [MJ/m2/hour]
    # 1.366 kW m^-2 * 1000 = 1366 W m^-2 = 1366 J s^-1 m^-2 * 3600 s hr^-1 *10^-6 = 4.9176 MJ m^-2 hr^-1
    Gsc = 4.92

    # Extra-terrestrial (TOA) SW radiation, Ra [MJ/m2/day] = f(Gsc [MJ/m2/hour],
    # dr [-], ws [rad], Lat [rad], decl [rad]), Allen et al 2005 equation 48. SW solar radiation in the absence of an atmosphere. Well-behaved function of the day of the year, time of day, lat and lon.
    Ra = ((24. / np.pi) * Gsc * dr * (ws * np.sin(Lat * np.pi/180) * np.sin(decl) + np.cos(Lat * np.pi/180) * np.cos(decl) * np.sin(ws))).rename({'day':'time'})
    Ra['time']=tasmax['time']
    
    return Ra

def netRadiation(rsds, elev, tasmax, tasmin, ea, lat_name='lat'):
    # Net SW radiation downward, Rns [MJ/m2/day] = f(alb [-], Rs [MJ/m2/day]), Allen et al 2005 equaton 43
    # Albedo of reference crop, alb [-]
    # Allen et al 205 equation 43 - constant value for daily and hourly.
    alb = 0.23

    # Stefan-Boltzmann constant, sigma [MJ/K4/m2/day]
    # SB constant = 5.67*10^-8 J s^-1 m^-2 K^-4 *10^-6 J->MJ * 86400 s day^-1 = 4.8988 MJ K^-4 m^-2 day^-1
    # Allen et al 2005 equation 44 sigma = 2.42 * 10^-10 MJ K^-4 m^-2 h^-1
    sigma = 4.901e-9

    _jday = rsds.time.dt.dayofyear.values
    jday = xr.DataArray(_jday, coords=[('day', _jday)], dims=["day"])
    
    # Downwelling SW radiation Rs [MJ/m2/day] = f(rsds [W/m2])
    # 24 hrs/day * 60 mins/hr * 60 sec/min = 86400 sec/day
    # J -> MJ => * 10^-6
    # rsds  = W m^-2 = J m^-2 s^-1 * 86400 s day^-1 * 10^-6 MJ J^-1 = MJ m^-2 day^-1
    Rs = rsds * 86400. / 1.e6
    Rns = (1. - alb) * Rs
                
    # Declination, decl [rad] = f(pi [-], jday [-])
    # Allen et al 2005 equation 51 solar declination
    decl = 0.409 * np.sin((2. * np.pi * jday / 365) - 1.39)
        
    # Sunset hour angle, ws [rad] = f(Lat [rad], decl [rad])
    # Allen et al 2005 equation 59 - is this correct? should it be tan of lat???
    Lat  = Rns[lat_name]
    ws = np.arccos(-1 * np.sin(np.deg2rad(Lat)) * np.tan(decl))
        
    # Inverse relative distance of earth from sun, dr [-] = f(pi [-], jday [-]), Allen et al 2005 equation 50
    dr = 1. + 0.033 * np.cos(2. * np.pi * jday / 365)

    # Solar constant, Gsc [MJ/m2/hour]
    # 1.366 kW m^-2 * 1000 = 1366 W m^-2 = 1366 J s^-1 m^-2 * 3600 s hr^-1 *10^-6 = 4.9176 MJ m^-2 hr^-1
    Gsc = 4.92

    # Extra-terrestrial (TOA) SW radiation, Ra [MJ/m2/day] = f(Gsc [MJ/m2/hour],
    # dr [-], ws [rad], Lat [rad], decl [rad]), Allen et al 2005 equation 48. SW solar radiation in the absence of an atmosphere. Well-behaved function of the day of the year, time of day, lat and lon.
    Ra = (24. / np.pi) * Gsc * dr * (ws * np.sin(Lat * np.pi/180) * np.sin(decl) + np.cos(Lat * np.pi/180) * np.cos(decl) * np.sin(ws))
    
    # Clear-sky SW radiation at surface, Rso [MJ/m2/day] = f(Ra [MJ/m2/day], Elev [m]), Allen et al 2005 equation 47
    Rso = Ra * (0.75 + 2.e-5 * elev)
    
    # Relative shortwave radiation, RsRso = f(Rs [MJ/m2/day], Rso [MJ/m2/day])
    # Allen et al 2005 limited to between 0.3 and 1.0
    # Resolve misaligned units 'day' and 'time'. Align Rso using the 'time' dimension from Rs.
######
    # Rso_aligned = Rso.reindex(time=Rs['time'])
    Rso = Rso.rename({'day':'time'})
    Rso['time']=Rs['time']
    
    # Perform the computation of RsRso
    RsRso = Rs / Rso
#######    
    # Clip values between 0.3 and 1.0
    RsRso = RsRso.clip(min=0.3, max=1.0)

    # Cloudiness function, fcd [-] = f(swratio [-]), See Allen et al 2005 equation 44/45 - limited to between 0.05 and 1.0, dimensionless
    fcd = 1.35 * RsRso - 0.35
    # Clip values between 0.05 and 1.0
    fcd = fcd.clip(min=0.05, max=1.0)
        
    # Net LW radiation upward, Rnl [MJ/m2/day] = f(sigma [MJ/K4/m2/day],
    # fcd [-], ea [kPa], Tmx [degC], Tmn [degC])
    # Net LW radiation is the difference between the LW radiation radiated upward from the standardised surface and LW radiation radiated downward from the atmospere. This method was introduced by Brunt (1932, 1952) and uses near surface vapour pressure to predict net surface emissivity. Allen et al 2005 equation 44
    Rnl = sigma * fcd * (0.34 - 0.14 * ea**0.5) * 0.5 * ((tasmax+273.16)**4. + (tasmin+273.16)**4.)
    
    # Net radiation, Rn [MJ/m2/day] = f(Rns [MJ/m2/day], Rnl [MJ/m2/day])
    # Allen et al 2005 equation 42
    Rn = Rns - Rnl
    return Rn
    
def wind2m(sfcWind,scaling_factor=None):
    if scaling_factor:
        # Use scaling factors from gridded roughness map for Australia based on observational station data (Zhang et al., 2022) to convert 10m wind speed datasets to 2m
        u2 = sfcWind * scaling_factor
    else:
        # Wind speed at 2 m, u2 [m/sec] = f(sfcWind [m/sec])
        # Log wind profile to 2 m from 10 m uses Allen et al 2005 equation 33, which is for wind measurements taken above a short grass or similar surface. Eqn B.14 in Appendix B is given for wind measured above alfalfa or similar vegetation having about 0.5 m height. Wind speed data collected above 2m are acceptable for use following adjustment to 2 m, and may be preferred if vegetation commonly exceeds 0.5 m. Measurement at a greater height reduces the influence of the taller vegetation.
        u2 = sfcWind * 4.87 / np.log(67.8 * 10. - 5.42) 
    return u2


def meanTemperature(tasmax,tasmin):        
    # Air temperature at 2 m, T [degC] = f(Tmx [degC], Tmn [degC])
    # This gives approximation of daily mean 2 m T. As stated by Allen et al 2005, this is the preferred method to calculate the mean, rather than an average of hourly T measurements, to provide consistency across all data sets.
    tas = (tasmax + tasmin) / 2.
    
    return tas

def satVapourPressure(tasmax, tasmin):
    # Saturated vapor pressure, esat [kPa] = f(Tmx [degC], Tmn [degC])
    # See Chapter 3 - Meteorological Data for formulae, Allen 2005, Equations 6 and 7
    # esat = 0.6108 exp{(17.27T/(T+237.3)}
    esat = (0.6108 * np.exp(17.27 * tasmax / (tasmin + 237.3)) + 0.6108 * np.exp(17.27 * tasmin / (tasmin + 237.3))) / 2.
    return esat

def actVapourPressure(esat, psl=None, huss=None, hurs=None):
    # Check that either 'hurs' is provided or both 'psl' and 'huss' are provided
    if (hurs is None and (psl is None or huss is None)):
        raise ValueError("Provide either 'hurs', or both 'psl' and 'huss'.")

    if hurs is not None:
        # Calculate ea from hurs (Relative Humidity in %) and esat
        ea = hurs * esat / 100  # Convert hurs from percentage to fraction and calculate ea
        
    elif psl is not None and huss is not None:
        # Specific humidity at 2 m (huss)
        w = huss / (1. - huss)
        
        # Actual vapor pressure (ea) = f(psl [Pa], w [kg/kg])
        ea = 0.001 * psl * w / (0.622 + w)  # Convert psl from Pa to kPa
        
    # Clip ea values to below or equal to esat
    ea = ea.clip(max=esat)
    
    return ea 

def calculate_FAO56_pmpet(Rn,t,u2,esat,ea,ps):
        
    # Define physical parameter constants:
    # See Allen at el 2005 for details of the parameters needed for this calculation
    print("Files read. Start calculation...")
    # Numerator constants, Cn_alf and Cn_grs [mm.K.sec/(day.m.kPa)], referred to in ASCE-EWRI as units of [K.mm.sec3/(Mg.day)], which are the same.
    Cn_alf = 1600.    # for tall reference crop (alfalfa) on a daily basis
    Cn_grs = 900.     # for short reference crop (grass) on a daily basis
        
    # Denominator constants, Cd_alf and Cd_grs [sec/m], equivalent to a wind function parameter
    Cd_alf = 0.38     # for tall reference crop (alfalfa) on a daily basis
    Cd_grs = 0.34     # for short reference crop (grass) on a daily basis
        
    # Psychometric constant, gamma [kPa/degC] = f(Pa [kPa])
    gamma = 0.000665 * ps * 10**(-3)

    # Ground heat flux, G [MJ/m2/day], assumed to be zero at the daily time-step
    G = 0.
        
    # Slope of the saturated vapor pressure curve at T,
    # delta [kPa/degC] = f(T [degC])
    # See Chapter 3 - Meteorological Data for formulae, Allen 2005, Equation 5    
    delta = 2503. * np.exp(17.27 * t / (t + 237.3)) / (t + 237.3)**2.

    # ASCE-EWRI ETrs, ETrs [mm/day] = f(0.408 [mm.m2/MJ], delta [kPa/K],
    # gamma [kPa/K], u2 [m/sec], Rn [MJ/m2/day], G [MJ/m2/day], T [degC],
    # esat [kPa], ea [kPa], Cn_alf [K.mm.sec3/Mg/day], Cd_alf [sec/m])
    ETrs = (0.408 * delta * (Rn - G) + gamma * (Cn_alf / (t + 273.)) * u2 * (esat - ea)) / (delta + gamma * (1. + Cd_alf * u2))
        
    # ASCE-EWRI ETos, ETos [mm/day] = f(0.408 [mm.m2/MJ], delta [kPa/K],
    # gamma [kPa/K], u2 [m/sec], Rn [MJ/m2/day], G [MJ/m2/day], T [degC],
    # esat [kPa], ea [kPa], Cn_grs [K.mm.sec3/Mg/day], Cd_grs [sec/m])
    ETos = (0.408 * delta * (Rn - G) + gamma * (Cn_grs / (t + 273.)) * u2 * (esat - ea)) / (delta + gamma * (1. + Cd_grs * u2))

    # Attributes
    ETrs.attrs['units'] = 'mm'
    ETrs.attrs['long_name'] = 'ASCE-EWRI tall reference crop ET for 0.5 m alfalfa [mm/day]'
    ETos.attrs['units'] = 'mm'
    ETos.attrs['long_name'] = 'ASCE-EWRI short reference crop ET for 12 cm grass [mm/day]'
    
    # List of coordinates to drop    
    coords_to_drop = ['height', 'level_height', 'model_level_number', 'sigma']
    # Drop coordinates only if they are present in the dataset
    ETrs = ETrs.drop([coord for coord in coords_to_drop if coord in ETrs.coords]).rename('ETrs').to_dataset()
    ETos = ETos.drop([coord for coord in coords_to_drop if coord in ETos.coords]).rename('ETos').to_dataset()

    ET = xr.merge([ETrs, ETos])
    # if 'rlat' in ET.dims:
    #     ET = ET.drop(['lat','lon']).rename_dims({'rlat': 'lat','rlon': 'lon'})
        
    ET.attrs['description'] = 'Uses daily or monthly data from downscaled CMIP6 outputs to calculate mean daily ETrs and ETos, using the version of the Penman-Monteith expression recommended by the American Society of Civil Engineers Environmental Water Resources Institute (ACSCE-EWRI), and known as the ASCE Standardized Reference ET equation. Reference: ASCE-EWRI (2005): The ASCE Standardized Reference Evapotranspiration Equation, Task Committee on Standardization of Reference Evapotranspiration, Ed. Richard G. Allen'
    return ET

def hargreaves_pet(tmin, tmax, tmean, ra):
    """
    Calculate Potential Evapotranspiration (PET) using the Hargreaves equation.

    Parameters:
    -----------
    tmin : float or array-like
        Minimum temperature (°C)
    tmax : float or array-like
        Maximum temperature (°C)
    tmean : float or array-like
        Mean temperature (°C)
    ra : float or array-like
        Extraterrestrial radiation (MJ/m²/day)

    Returns:
    --------
    pet : float or array-like
        Potential Evapotranspiration (mm/day)
    """
    pet = 0.0023 * (tmean + 17.8) * ((tmax - tmin) ** 0.5) * ra
    pet.attrs['units'] = 'mm'
    pet.attrs['long_name'] = 'Temperature-based Hargraves PET [mm/day]'
    return pet.rename('pet_hargraves').to_dataset()


def convertTemperature(temp):
    if temp.attrs['units'] not in ('degC', 'K'):
        raise ValueError("The 'unit' argument for tasmax and tasmin is neither 'C' (Celsius) or 'K' (Kelvin).")
    if temp.attrs['units'] == 'K':
        temp = temp - 273.16
        temp.attrs['units'] = 'degCC'
    return temp
        

In [23]:
startY = 2011
endY = 2014

pet_method = 'Hargraves' #'FAO56
freq = 'day'#'month' #'day'

out_dir = '/g/data/mn51/projects/work_package_4/climate_hazard_indices/drought/'

#< Need to regrid: ratio_10mto2m = xr.open_dataset('/g/data/mn51/projects/work_package_4/climate_hazard_indices/drought/ratio_10mto2m.nc').ratio

In [25]:
%%time
root_dir = '/g/data/ia39/australian-climate-service/release/CORDEX/output-Adjust/CMIP6/bias-adjusted-input/AUST-05i/'#'/g/data/ia39/australian-climate-service/release/CORDEX/output-Adjust/CMIP6/bias-adjusted-input/AUST-05i/'
indir = root_dir+'BOM/ACCESS-ESM1-5/historical/r6i1p1f1/BARPA-R/v1-r1/'

if pet_method == 'FAO56':
    elev = xr.open_dataset(f'{indir}fx/orog/latest/orog_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_fx.nc').orog

for year in range(startY,endY+1):
    # Start the lap timer
    start_time = time.time()

    print(f'#################### Processing year {year}... ####################')
    if pet_method == 'FAO56':
        et_filename = f'{out_dir}pet_fao/PET_FAO_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_{freq}_{year}0101-{year}1231.nc'
    else:
        et_filename = f'{out_dir}pet_Hargraves/PET_Hargraves_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_{freq}_{year}0101-{year}1231.nc'
    
    if os.path.exists(et_filename)==False:
        def _preprocess(ds):
            return ds.sel(time=slice(f'{startY}-01-01',f'{endY}-12-31')).chunk({'time':30,'lat':-1,'lon':-1})
    
        #< Read input data
        tasmax = xr.open_mfdataset(f'{indir}day/tasmax/v20241216/tasmax_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmax
        tasmin = xr.open_mfdataset(f'{indir}day/tasmin/v20241216/tasmin_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmin

        if pet_method == 'FAO56':
            rsds = xr.open_mfdataset(f'{indir}day/rsds/v20231001/rsds_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).rsds
            huss = xr.open_mfdataset(f'{indir}day/huss/latest/huss_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).huss
            hurs = xr.open_mfdataset(f'{indir}day/hurs/latest/hurs_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).hurs
            psl = xr.open_mfdataset(f'{indir}day/psl/latest/psl_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).psl
            sfcWind = xr.open_mfdataset(f'{indir}day/sfcWind/latest/sfcWind_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).sfcWind
    
        #< Calculate PET input variables
        if tasmax.attrs['units'] == 'K':
            tasmax = convertTemperature(tasmax)
        if tasmin.attrs['units'] == 'K':
            tasmin = convertTemperature(tasmin)
        t = meanTemperature(tasmax,tasmin)

        if pet_method == 'FAO56':
            esat = satVapourPressure(tasmax, tasmin)
            ea = actVapourPressure(esat, hurs=hurs)
            u2 = wind2m(sfcWind,scaling_factor=None)
            Rn = netRadiation(rsds, elev, tasmax, tasmin, ea)
        else:
            Ra = extraTerrRadiation(tasmax)        
    
        #< Calculate PET
        if pet_method == 'FAO56':
            ET = calculate_FAO56_pmpet(Rn,t,u2,esat,ea,psl).astype(np.float32)
        else:
            ET = hargreaves_pet(tasmin, tasmax, t, Ra).astype(np.float32)

        if freq == 'month':
            ET = ET.resample(time='M').mean()
    
        #< Write to file
        if pet_method == 'FAO56':
            ET.to_netcdf(et_filename,encoding=enc)
        else:
            ET.to_netcdf(et_filename,encoding=enc_H)
    
    else:
        print(f'PET_FAO file for year {year} exists in output directory.')

    
    # End the timer
    print(f"Time taken: {(time.time()- start_time):.2f} seconds") 


#################### Processing year 2011... ####################
Time taken: 66.31 seconds
#################### Processing year 2012... ####################
Time taken: 69.05 seconds
#################### Processing year 2013... ####################
Time taken: 72.75 seconds
#################### Processing year 2014... ####################
Time taken: 55.85 seconds
CPU times: user 1min 10s, sys: 13.4 s, total: 1min 24s
Wall time: 4min 23s


## PET calculation for raw BOM-BARPA

In [25]:
%%time
root_dir = '/g/data/ia39/australian-climate-service/release/CORDEX/output-Adjust/CMIP6/bias-adjusted-input/AUST-05i/'
indir = root_dir+'BOM/ACCESS-ESM1-5/historical/r6i1p1f1/BARPA-R/v1-r1/'

if pet_method == 'FAO56':
    elev = xr.open_dataset(f'{indir}fx/orog/latest/orog_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_fx.nc').orog

for year in range(startY,endY+1):
    # Start the lap timer
    start_time = time.time()

    print(f'#################### Processing year {year}... ####################')
    if pet_method == 'FAO56':
        et_filename = f'{out_dir}pet_fao/PET_FAO_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_{freq}_{year}0101-{year}1231.nc'
    else:
        et_filename = f'{out_dir}pet_Hargraves/PET_Hargraves_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_{freq}_{year}0101-{year}1231.nc'
    
    if os.path.exists(et_filename)==False:
        def _preprocess(ds):
            return ds.sel(time=slice(f'{startY}-01-01',f'{endY}-12-31')).chunk({'time':30,'lat':-1,'lon':-1})
    
        #< Read input data
        tasmax = xr.open_mfdataset(f'{indir}day/tasmax/v20241216/tasmax_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmax
        tasmin = xr.open_mfdataset(f'{indir}day/tasmin/v20241216/tasmin_AUST-05i_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmin

        if pet_method == 'FAO56':
            rsds = xr.open_mfdataset(f'{indir}day/rsds/v20231001/rsds_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).rsds
            huss = xr.open_mfdataset(f'{indir}day/huss/latest/huss_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).huss
            hurs = xr.open_mfdataset(f'{indir}day/hurs/latest/hurs_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).hurs
            psl = xr.open_mfdataset(f'{indir}day/psl/latest/psl_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).psl
            sfcWind = xr.open_mfdataset(f'{indir}day/sfcWind/latest/sfcWind_AUS-15_ACCESS-ESM1-5_historical_r6i1p1f1_BOM_BARPA-R_v1-r1_day_{year}01-{year}12.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).sfcWind
    
        #< Calculate PET input variables
        if tasmax.attrs['units'] == 'K':
            tasmax = convertTemperature(tasmax)
        if tasmin.attrs['units'] == 'K':
            tasmin = convertTemperature(tasmin)
        t = meanTemperature(tasmax,tasmin)

        if pet_method == 'FAO56':
            esat = satVapourPressure(tasmax, tasmin)
            ea = actVapourPressure(esat, hurs=hurs)
            u2 = wind2m(sfcWind,scaling_factor=None)
            Rn = netRadiation(rsds, elev, tasmax, tasmin, ea)
        else:
            Ra = extraTerrRadiation(tasmax)        
    
        #< Calculate PET
        if pet_method == 'FAO56':
            ET = calculate_FAO56_pmpet(Rn,t,u2,esat,ea,psl).astype(np.float32)
        else:
            ET = hargreaves_pet(tasmin, tasmax, t, Ra).astype(np.float32)

        if freq == 'month':
            ET = ET.resample(time='M').mean()
    
        #< Write to file
        if pet_method == 'FAO56':
            ET.to_netcdf(et_filename,encoding=enc)
        else:
            ET.to_netcdf(et_filename,encoding=enc_H)
    
    else:
        print(f'PET_FAO file for year {year} exists in output directory.')

    
    # End the timer
    print(f"Time taken: {(time.time()- start_time):.2f} seconds") 


#################### Processing year 2011... ####################
Time taken: 66.31 seconds
#################### Processing year 2012... ####################
Time taken: 69.05 seconds
#################### Processing year 2013... ####################
Time taken: 72.75 seconds
#################### Processing year 2014... ####################
Time taken: 55.85 seconds
CPU times: user 1min 10s, sys: 13.4 s, total: 1min 24s
Wall time: 4min 23s


## PET calculation for raw CSIRO-CCAM

In [5]:
%%time
indir = '/g/data/hq89/CCAM/output/CMIP6/DD/AUS-10i/CSIRO/ACCESS-ESM1-5/historical/r6i1p1f1/CCAM-v2203-SN/v1-r1/'
elev = xr.open_dataset(f'{indir}fx/orog/v20231206/orog_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1.nc').orog

for year in range(startY,endY+1):
    # Start the lap timer
    start_time = time.time()

    print(f'#################### Processing year {year}... ####################')
    et_filename = f'/g/data/mn51/projects/work_package_4/climate_hazard_indices/drought/pet_fao/PET_FAO_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}01-{year}12.nc'

    if os.path.exists(et_filename)==False:
        def _preprocess(ds):
            return ds.sel(time=slice(f'{startY}-01-01',f'{endY}-12-31')).chunk({'time':30,'lat':-1,'lon':-1})
    
        #< Read input data
        print('rsds')
        rsds = xr.open_mfdataset(f'{indir}day/rsds/v20231206/rsds_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).rsds
        print('tasmax')
        tasmax = xr.open_mfdataset(f'{indir}day/tasmax/v20231206/tasmax_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmax
        print('tasmin')
        tasmin = xr.open_mfdataset(f'{indir}day/tasmin/v20231206/tasmin_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmin
        # huss = xr.open_mfdataset(f'{indir}day/huss/latest/huss_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).huss
        print('hurs')
        hurs = xr.open_mfdataset(f'{indir}day/hurs/v20231206/hurs_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).hurs
        print('psl')
        psl = xr.open_mfdataset(f'{indir}day/psl/v20231206/psl_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).psl
        print('sfcWind')
        sfcWind = xr.open_mfdataset(f'{indir}day/sfcWind/v20240327/sfcWind_AUS-10i_ACCESS-ESM1-5_historical_r6i1p1f1_CSIRO_CCAM-v2203-SN_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).sfcWind
    
        #< Calculate PET input variables
        tasmax = convertTemperature(tasmax)
        tasmin = convertTemperature(tasmin)
        esat = satVapourPressure(tasmax, tasmin)
        ea = actVapourPressure(esat, hurs=hurs)
        t = meanTemperature(tasmax,tasmin)
        u2 = wind2m(sfcWind,scaling_factor=None)
        Rn = netRadiation(rsds, elev, tasmax, tasmin, ea)
    
        #< Calculate PET
        ET = calculate_ASCE_pmpet(Rn,t,u2,esat,ea,psl).astype(np.float32)
    
        #< Write to file
        ET.to_netcdf(et_filename,encoding=enc)
    
    else:
        print(f'PET_FAO file for year {year} exists in output directory.')

    
    # End the timer
    print(f"Time taken: {(time.time()- start_time):.2f} seconds") 


#################### Processing year 1995... ####################
PET_FAO file for year 1995 exists in output directory.
Time taken: 0.02 seconds
#################### Processing year 1996... ####################
PET_FAO file for year 1996 exists in output directory.
Time taken: 0.00 seconds
#################### Processing year 1997... ####################
PET_FAO file for year 1997 exists in output directory.
Time taken: 0.00 seconds
#################### Processing year 1998... ####################
PET_FAO file for year 1998 exists in output directory.
Time taken: 0.00 seconds
#################### Processing year 1999... ####################
PET_FAO file for year 1999 exists in output directory.
Time taken: 0.00 seconds
#################### Processing year 2000... ####################
PET_FAO file for year 2000 exists in output directory.
Time taken: 0.00 seconds
#################### Processing year 2001... ####################
PET_FAO file for year 2001 exists in output directory.
Tim

## PET calculation for raw NSW-WRF

In [14]:
%%time
indir = '/g/data/zz63/NARCliM2-0/output/CMIP6/DD/AUS-18/NSW-Government/ACCESS-ESM1-5/historical/r6i1p1f1/NARCliM2-0-WRF412R3/v1-r1/'
elev = xr.open_dataset(f'{indir}fx/orog/latest/orog_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_fx.nc').orog

for year in range(startY,endY+1):
    # Start the lap timer
    start_time = time.time()

    print(f'#################### Processing year {year}... ####################')
    et_filename = f'/g/data/mn51/projects/work_package_4/climate_hazard_indices/drought/pet_fao/PET_FAO_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc'

    if os.path.exists(et_filename)==False:
        def _preprocess(ds):
            return ds.sel(time=slice(f'{startY}-01-01',f'{endY}-12-31')).chunk({'time':365,'rlat':-1,'rlon':-1})
    
        #< Read input data
        rsds = xr.open_mfdataset(f'{indir}day/rsds/latest/rsds_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).rsds
        tasmax = xr.open_mfdataset(f'{indir}day/tasmax/latest/tasmax_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmax
        tasmin = xr.open_mfdataset(f'{indir}day/tasmin/latest/tasmin_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).tasmin
        # huss = xr.open_mfdataset(f'{indir}day/huss/latest/huss_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).huss
        hurs = xr.open_mfdataset(f'{indir}day/hurs/latest/hurs_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).hurs
        psl = xr.open_mfdataset(f'{indir}day/psl/latest/psl_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).psl
        sfcWind = xr.open_mfdataset(f'{indir}day/sfcWind/latest/sfcWind_AUS-18_ACCESS-ESM1-5_historical_r6i1p1f1_NSW-Government_NARCliM2-0-WRF412R3_v1-r1_day_{year}0101-{year}1231.nc',combine='nested',concat_dim='time',parallel=True,preprocess=_preprocess).sfcWind
    
        #< Calculate PET input variables
        tasmax = convertTemperature(tasmax)
        tasmin = convertTemperature(tasmin)
        esat = satVapourPressure(tasmax, tasmin)
        ea = actVapourPressure(esat, hurs=hurs)
        t = meanTemperature(tasmax,tasmin)
        u2 = wind2m(sfcWind,scaling_factor=None)
        Rn = netRadiation(rsds, elev, tasmax, tasmin, ea, lat_name='rlat')
    
        #< Calculate PET
        ET = calculate_ASCE_pmpet(Rn,t,u2,esat,ea,psl).astype(np.float32)
    
        #< Write to file
        ET.to_netcdf(et_filename,encoding=enc)
    
    else:
        print(f'PET_FAO file for year {year} exists in output directory.')

    
    # End the timer
    print(f"Time taken: {(time.time()- start_time):.2f} seconds") 


#################### Processing year 1995... ####################
Files read. Start calculation...
Time taken: 31.62 seconds
#################### Processing year 1996... ####################
Files read. Start calculation...
Time taken: 32.48 seconds
#################### Processing year 1997... ####################
Files read. Start calculation...
Time taken: 36.85 seconds
#################### Processing year 1998... ####################
Files read. Start calculation...
Time taken: 30.55 seconds
#################### Processing year 1999... ####################
Files read. Start calculation...
Time taken: 32.57 seconds
#################### Processing year 2000... ####################
Files read. Start calculation...
Time taken: 30.71 seconds
#################### Processing year 2001... ####################
Files read. Start calculation...
Time taken: 32.50 seconds
#################### Processing year 2002... ####################
Files read. Start calculation...
Time taken: 34.82 seconds
